In [8]:
# Implementation of Gaussian Process for utility inference using Pairwise Comparison 
# Details see:
#   1. "A Tutorial on Bayesian Optimization of Expensive Cost Functions"
#   2. "Preference learning with Gaussian processes" (ICML 2005)
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import RandomState
from scipy.stats import norm
import copy

ModuleNotFoundError: No module named 'gpy'

In [2]:
class Dataset:
    """Dataset class
    
    Attributes:
        datapoints: matrix for datapoints (dimension t x n)
        comparisons: matrix for comparisons (dimension m x 2)
    """
    
    def __init__(self, num_features):
        """Inits Dataset with the number of features"""
        self.datapoints = np.empty((0, num_features))
        self.comparisons = np.empty((0, 2), dtype=np.int)

    def add_single_comparison(self, sup, inf):
        """Adds a single comparison to the dataset.
        
        Args:
            sup: The datapoint index that is superior in the comparison.
            inf: The datapoint index that is inferior in the comparison.
        """
        # add superior and inferior to our datapoints and get the indices in dataset
        sup_idx = self._add_single_datapoint(sup)
        inf_idx = self._add_single_datapoint(inf)
        self.comparisons = np.vstack((self.comparisons, [sup_idx, inf_idx]))
        
        return True

    def _add_single_datapoint(self, new_datapoint):
        """Adds a single datapoint to the existing dataset and returns index.
        
        Args:
            new_datapoint: The new datapoint to be added.        
            
        Returns:
            x_new_idx: The index of the new datapoint in the existing dataset.
        """
        self.datapoints = np.vstack((self.datapoints, new_datapoint))
        new_datapoint_index = self.datapoints.shape[0] - 1
        return new_datapoint_index

    def get_index(self, datapoint):
        """Gets the index of a datapoint in the dataset.
        
        Args:
            datapoint: A single datapoint.
        
        Returns:            
            The index of datapoint in the dataset.
        """
        return np.argmax(np.sum(datapoint != self.datapoints, axis=1) == 0)

In [3]:
class GP_pairwise:
    """Gaussian process with a discrete-choice probit model (latent utility function).
    
    Attributes:
        sigma: Hyperparameter for std of the normal distributed noise for the utility function.
        theta: Hyperparameter for kernel width.
        seed: Seed for random state.
        datapoints: Matrix for the observed data.
        comparisons: Matrix for comparisons of the observed data.
        f_map: Approximate utility values of the datapoints.
        K: Covariance matrix of datapoints.
        K_inv: Inverse of the covariance matrix of datapoints.
        C: The matrix C for observed data.
        C_inv: The inverse of matrix C for observed data.
    """
    
    def __init__(self, sigma=0.01, theta=50, seed=None):
        """Inits GP class with all attributes."""
        self.sigma = sigma
        self.theta = theta
        self.random_state = RandomState(seed)
        self.datapoints = None
        self.comparisons = None
        self.f_map = None
        self.K = None
        self.K_inv = None
        self.C = None
        self.C_inv = None

    def update(self, dataset):
        """Update the Gaussian process using the given data.
        
        Args:
            dataset: Dataset consists of datapoints and comparison matrix data 
        """
        self.datapoints = dataset.datapoints
        self.comparisons = dataset.comparisons

        # compute the covariance matrix and its inverse
        self.K = self._get_K(self.datapoints)
        self.K_inv = self._get_inv(self.K)

        # compute the MAP estimate of f
        self.f_map = self._get_f_map()

        # compute C matrix given f_MAP and its inverse (psudo-inverse)
        self.C = self._get_C()
        self.C_inv = self._get_inv(self.C)

        return True 
    
    def sample(self, sample_points):
        """Get a sample from the current GP at the given points.
        
        Args:
            sample_points: The points at which we want to take the sample.
            
        Returns: 
            The values of the GP sample at the input points.
        """
        # get the mean and the variance of the predictive (multivariate gaussian) distribution at the sample points
        mean, var = self.get_Gaussian_params(sample_points, pointwise=False)

        # sample from the multivariate gaussian with the given parameters
        f_sample = self.random_state.multivariate_normal(mean, var, 1)[0]

        return f_sample
    
    def predict(self, sample_point):
        """Predicts value function for a single datapoint"""
        mean, var = self.get_Gaussian_params(np.array([sample_point]), pointwise=False)
        f_sample = self.random_state.multivariate_normal(mean, var, 1)[0]
        return mean

    def get_Gaussian_params(self, x_new, pointwise):
        """Gets the Gaussian parameters.
        
        Args:
            x_new: the points for which we want the predictive parameters.
            pointwise: whether we want pointwise variance or the entire covariance matrix.
            
        Returns:
            The predictive parameters of the Gaussian distribution at the given datapoints.
        """
        # if we don't have any data yet, use prior GP to make predictions
        if self.datapoints is None or self.f_map is None:
            pred_mean, pred_var = self._evaluate_prior(x_new)

        # otherwise compute predictive mean and covariance
        else:
            k_T = self._get_K(x_new, self.datapoints, noise=False)
            k = self._get_K(self.datapoints, x_new, noise=False)
            k_plus = self._get_K(x_new, noise=False)
            pred_mean = self._prior_mean(k_plus) + np.dot(np.dot(k_T, self.K_inv),
                                                         (self.f_map - self._prior_mean(self.datapoints)))            
            pred_var = k_plus - np.dot(np.dot(k_T, self._get_inv(self.K + self.C_inv)), k)
        if pointwise:
            pred_var = pred_var.diagonal()

        return pred_mean, pred_var

    
    def _get_K(self, x1, x2=None, noise=True):
        """Computes covariance matrix for preference data using the kernel function.
        
        Args:
            x1: The datapoints for which to compute covariance matrix.
            x2: If None, covariance matrix will be square for the input x1,
                If not None, covariance will be between x1 (rows) and x2 (cols)
            noise: Whether to add noise to the diagonal of the covariance matrix.
            
        Returns:
            The covariance matrix K.            
        """
        if x2 is None:
            x2 = x1
        else: 
            noise = False
        K = self._k(np.repeat(x1, x2.shape[0], axis=0), 
                               np.tile(x2, (x1.shape[0], 1)))
        K = K.reshape((x1.shape[0], x2.shape[0]))
        if noise:
            K += self.sigma ** 2 * np.eye(K.shape[0])
        return K
    
    def _k(self, x1, x2):
        """Exponentiated quadratic kernel function"""
        k = 0.8**2 * np.exp(-(1. / (2. * (self.theta ** 2))) * np.linalg.norm(x1 - x2, axis=1) ** 2)
        return k
        
    def _get_f_map(self):
        """Computes maximum a posterior (MAP) evaluation of f given the data using Newton's method
        
        Returns: 
            MAP of the Gassian processes values at current datapoints
        """
        converged = False
        try_no = 0

        f_map = None

        # Newton's method to approximate f_MAP
        while not converged and try_no < 1:

            # randomly initialise f_map
            f_map = self.random_state.uniform(0., 1., self.datapoints.shape[0])

            for m in range(100):
                # compute Z
                f_sup = np.array([f_map[self.comparisons[i, 0]] for i in range(self.comparisons.shape[0])])
                f_inf = np.array([f_map[self.comparisons[i, 1]] for i in range(self.comparisons.shape[0])])
                Z = self._get_Z(f_sup, f_inf)
                Z_logpdf = norm.logpdf(Z)
                Z_logcdf = norm.logcdf(Z)

                # compute b
                b = self._get_b(Z_logpdf, Z_logcdf)

                # compute gradient g
                g = self._get_g(f_map, b)

                # compute hessian H
                C = self._get_C(Z)
                H = - self.K_inv + C
                H_inv = self._get_inv(H)

                # perform update
                update = np.dot(H_inv, g)
                f_map -= update

                # stop criterion
                if np.linalg.norm(update) < 0.0001:
                    converged = True
                    break
                                   
            if not converged:
                print("Did not converge.")
                try_no += 1

        return f_map

    def _get_Z(self, f_sup, f_inf):
        """Gets the random variable Z based on given sup and inf pair."""
        return (f_sup - f_inf) / (np.sqrt(2) * self.sigma)
    
    def _get_b(self, Z_logpdf, Z_logcdf):
        """Gets the N-dimensional vector b"""
        h_j = np.array([np.array(self.comparisons[:, 0] == j, dtype=int) - np.array(self.comparisons[:, 1] == j, dtype=int) 
                        for j in range(self.datapoints.shape[0])])
        
        b = np.sum(h_j * np.exp(Z_logpdf - Z_logcdf), axis=1) / (np.sqrt(2) * self.sigma)
        return b
    
    def _get_g(self, f_map, b):
        """Gets the gradient g"""
        return -np.dot(self.K_inv, (f_map - self._prior_mean(self.datapoints))) + b

    def _get_C(self, Z=None):
        """Gets the matrix C"""
        if Z is None:
            # compute z
            f_sup = np.array([self.f_map[self.comparisons[i, 0]] for i in range(self.comparisons.shape[0])])
            f_inf = np.array([self.f_map[self.comparisons[i, 1]] for i in range(self.comparisons.shape[0])])
            Z = (f_sup - f_inf) / (np.sqrt(2) * self.sigma)

        Z_logpdf = norm.logpdf(Z)
        Z_logcdf = norm.logcdf(Z)

        # init with zeros
        C = np.zeros((self.datapoints.shape[0], self.datapoints.shape[0]))

        # build up diagonal for pairs (x, x)
        diag_arr = np.array([self._get_C_entry(m, m, Z, Z_logpdf, Z_logcdf) for m in
                             range(self.datapoints.shape[0])])
        np.fill_diagonal(C, diag_arr)  # happens in-place

        # go through the existing list of comparisons and update C
        for k in range(self.comparisons.shape[0]):
            m = self.comparisons[k, 0]  # superior
            n = self.comparisons[k, 1]  # inferior
            C[m, n] = self._get_C_entry(m, n, Z, Z_logpdf, Z_logcdf)
            C[n, m] = self._get_C_entry(n, m, Z, Z_logpdf, Z_logcdf)

        # add jitter terms to make matrix C positive semidefinite for stable computation
        C += np.eye(self.datapoints.shape[0]) * 0.01

        return C

    def _get_C_entry(self, m, n, Z, Z_logpdf, Z_logcdf):
        """Gets a single entry for the Hessian matrix at indices (m,n)"""
        h_x_m = np.array(self.comparisons[:, 0] == m, dtype=int) - np.array(self.comparisons[:, 1] == m, dtype=int)
        h_x_n = np.array(self.comparisons[:, 0] == n, dtype=int) - np.array(self.comparisons[:, 1] == n, dtype=int)
        p = h_x_m * h_x_n * (np.exp(2.*Z_logpdf - 2.*Z_logcdf) + Z * np.exp(Z_logpdf - Z_logcdf))
        c = - np.sum(p) / (2 * self.sigma**2)
        return c 
    
    
    def _evaluate_prior(self, input_points):
        """Evaluates the prior distribution given some datapoints.
        
        Args:
            input_points: input datapoints at which to evaluate prior distribution.
            
        Returns:
            The predictive mean and covariance at the given inputs.
        """
        pred_mean = self._prior_mean(input_points)
        num_inputs = input_points.shape[0]
        pred_cov = self._k(np.repeat(input_points, num_inputs, axis=0),
                           np.tile(input_points, (num_inputs, 1))).reshape((num_inputs, num_inputs))
        return pred_mean, pred_cov
                                             
    def _get_inv(self, M):
        """Computes the inverse of the given matrix or the psudoinverse."""
        try:
            M_inv = np.linalg.inv(M)
        except:
            M_inv = np.linalg.pinv(M)
        return M_inv
    
    def _prior_mean(self, x):
        """Returns the prior mean of zeros"""
        m = np.zeros(x.shape[0])
        return m

In [4]:
class AcquisitionFunction:
    def __init__(self, input_domain, seed):
        """ An acquirer for a discrete set of points.
        
        Attributes:
            input_domain: the datapoints on which the discrete acquirer is defined.
            random_state: based on the random seed.
        """
        self.input_domain = copy.deepcopy(input_domain)
        self.random_state = np.random.RandomState(seed)
        self.history = np.empty((0, self.input_domain.shape[1]))


    def get_next_point(self, dataset, gaussian_process):
        """Selects a single next datapoint to query.
        
        Args: 
            gaussian_process
            dataset
        """
        next_point_1, next_point_2 = self.get_next_point_VAR(dataset, gaussian_process)
        self.history = np.vstack((self.history, next_point_1, next_point_2))
        return next_point_1, next_point_2
    
    def get_next_point_VAR(self, dataset, gaussian_process):
        """Selects the next two datapoint to query using maximum uncertainty
        
        Args:
            dataset
            gaussian_process
            
        Returns:
            The optimal next pair based on maximum variance.
        """
        var = np.zeros(self.input_domain.shape[0])
        batch_size = 16
        for curr_idx in range(0, self.input_domain.shape[0]+batch_size, batch_size):
            var[curr_idx:curr_idx+batch_size] = self._get_variance(self.input_domain[curr_idx:curr_idx+batch_size], gaussian_process)
        # find the two points with the highest variance, and which can be queried next
        next_point1 = self.input_domain[np.argsort(var)[-1]]
        next_point2 = self.input_domain[np.argsort(var)[-2]]
        next_point_idx1, next_point_idx2 = 1, 2
        while self._check_duplicate(dataset, next_point1, next_point2):
            next_point1 = self.input_domain[np.argsort(-var)[next_point_idx1]]
            next_point2 = self.input_domain[np.argsort(-var)[next_point_idx2]]
            next_point_idx1 += 1
            next_point_idx2 += 1
        return next_point1, next_point2

    def _get_variance(self, datapoints, gaussian_process):
        """Obtains the predictive pointwise variance."""
        pred_var = gaussian_process.get_Gaussian_params(datapoints, pointwise=True)[1]
        return pred_var    
    
    def _check_duplicate(self, dataset, point1, point2):
        """Checks if there are duplicate pairs being sampled."""
        if point1 not in dataset.datapoints or point2 not in dataset.datapoints:
            return False 
        idx1, idx2 = np.where(dataset.datapoints==point1), np.where(dataset.datapoints==point2)
        return True if [idx1, idx2] or [idx2, idx1] in dataset.comparisons else False

In [5]:
class ComparisonOracle:
    def __init__(self, utility_function):
        self.utility_function = utility_function
        
    def compare(self, choice1, choice2):
        if self.utility_function(choice1) > self.utility_function(choice2):
            return choice1, choice2
        else:
            return choice2, choice1  

def sqare_root_utility(subset):
    return sum([x ** 0.5 for x in subset])

def cube_root_utility(subset):
    return sum([x ** (1./3) for x in subset])

def log_utility(subset):
    return sum([np.log(x) for x in subset])

def train(gp, af, max_q=50):
    for i in range(1, max_q+1): 
        print('-------------Train', i, '--------------')
        next1, next2 = af.get_next_point(dataset, gp)
        print('Optimal next pair is:', next1, next2)
        sup, inf = oracle.compare(next1, next2)
        dataset.add_single_comparison(sup, inf)
        print('User chooses:', sup)
        print('Updating GP with new data...')
        gp.update(dataset)
    print('---------Finished training!----------\n') 
    
def test(gp, af, test_size=50):
    gp_oracle = ComparisonOracle(gp.predict)
    success = 0
    for i in range(1, test_size+1):
        print('---------------Test', i, '---------------')
        test1 = np.random.randint(1, 50, 2)
        test2 = np.random.randint(1, 50, 2)
        gp_op = gp_oracle.compare(test1, test2)[0]
        oracle_op = oracle.compare(test1, test2)[0]
        print('Testing pair:', test1, test2)
        print('GP chooses', gp_op)
        print('User chooses', oracle_op)
        if np.array_equal(gp_op, oracle_op):
            success += 1
    print('-----------Finished testing!----------\n') 
    print("Accuracy: ", success/test_size)    

In [6]:
#### Simulation for pairwise-comparison experiment 
## Setup: 
##      - want to hire <= 100 people
##      - each subset is represented by [W, M] -> [number of women, number of men]
##      - capacity constraint: W + M <= 100
##      - value function defined on the subsets 
##      - true value function: square-root / cubic-root / log utility
## Goal:
##      - learn value function using GP + active learning (choosing optimal pair sequentially)
##      - Use as few query as possible (using 20 queries in the current example)

#### set seed for experiment

#### Create oracle w/ sqaure root utility, cubic or log utility
oracle = ComparisonOracle(cube_root_utility)

#### Create dataset object
dataset = Dataset(2)

#### Create query domain
input_domain = np.array([[x, 100 - x] for x in range(1, 100)])

#### GP + AF
gp = GP_pairwise(theta=20, seed=1) # build a new GP process
af = AcquisitionFunction(input_domain, seed=10) # build a new acquisition function

#### Training with active learning on 20 queries 
train(gp, af, 20)

#### Out-of-sample testing on 30 queries 
test(gp, af, 30)

-------------Train 1 --------------
Optimal next pair is: [99  1] [25 75]
User chooses: [25 75]
Updating GP with new data...
-------------Train 2 --------------
Optimal next pair is: [62 38] [61 39]
User chooses: [61 39]
Updating GP with new data...
-------------Train 3 --------------
Optimal next pair is: [62 38] [60 40]
User chooses: [60 40]
Updating GP with new data...
-------------Train 4 --------------
Optimal next pair is: [60 40] [63 37]
User chooses: [60 40]
Updating GP with new data...
-------------Train 5 --------------
Optimal next pair is: [63 37] [59 41]
User chooses: [59 41]
Updating GP with new data...
-------------Train 6 --------------
Optimal next pair is: [59 41] [64 36]
User chooses: [59 41]
Updating GP with new data...
-------------Train 7 --------------
Optimal next pair is: [64 36] [ 1 99]
User chooses: [64 36]
Updating GP with new data...
-------------Train 8 --------------
Optimal next pair is: [86 14] [85 15]
User chooses: [85 15]
Updating GP with new data...


In [ ]:
#### Visualization for utility functions
